In [53]:
#--Import Bibliotecas----------------------------------------------------------------------
from os import chdir, getcwd, listdir
from os.path import isfile
from datetime import datetime
import xmltodict, openpyxl
import pandas as pd

#--nomeArquivos para Acesso--------------------------------------------------------------------
xCaminhPlan = "C:\\Python_Bases\\BaseXML.xlsx"
xCaminhoXML = "C:\\Python_Bases\\XML_das_Notas\\"
#--Variavel Capsula------------------------------------------------------------------------
capsulaXML = []
#--Tabela pandas---------------------------------------------------------------------------
titulo = ['ChaveNF','dtEmisao','Volume','PesoB','Vprod','VtProd',
                  'emitEnd','emitCid','emitNome','emitCNPJ','emitIE',
                  'destNome','destCNPJ','destEnd','destCid','destIE',
                  'numerNF','digitação']
sTabela = pd.DataFrame(columns=titulo)
#------------------------------------------------------------------------------------------
#--Leitura do XML---------------------------------------------------------------------------
def extrair_XML(nomeArquivo):
    xmlInicio = ''
    
    print(xmlInicio)
    # # Caminho ref. ao arquivo---------------------------------------------------------------
    xPath = (xCaminhoXML+nomeArquivo)
    # lopp de carregamento------------------------------------------------------------------
    with open(xPath, errors='ignore') as f:
        xml = xmltodict.parse(f.read())      
        #---------------------------------------------------------------------
        emitNome = xml["nfeProc"]["NFe"]["infNFe"]["emit"]["xNome"]
        emitCNPJ = xml["nfeProc"]["NFe"]["infNFe"]["emit"]["CNPJ"]   
        emitEnd = xml["nfeProc"]["NFe"]["infNFe"]["emit"]["enderEmit"]["xLgr"]   
        emitCid = xml["nfeProc"]["NFe"]["infNFe"]["emit"]["enderEmit"]["xMun"]
        emitIE = xml["nfeProc"]["NFe"]["infNFe"]["emit"]["IE"]
        #--------------------------------------------------------------------
        destNome =  xml["nfeProc"]["NFe"]["infNFe"]["dest"]["xNome"] 
        destCNPJ = xml["nfeProc"]["NFe"]["infNFe"]["dest"]["CNPJ"]  
        destEnd = xml["nfeProc"]["NFe"]["infNFe"]["dest"]["enderDest"]["xLgr"]
        destCid = xml["nfeProc"]["NFe"]["infNFe"]["dest"]["enderDest"]["xMun"]   
        destIE = xml["nfeProc"]["NFe"]["infNFe"]["emit"]["IE"]     
        #--------------------------------------------------------------------
        numerNF = xml["nfeProc"]["NFe"]["infNFe"]["ide"]["nNF"]    
        dtEmisao = xml["nfeProc"]["NFe"]["infNFe"]["ide"]["dhEmi"]
        #--Converção de data --------------------------------------------------
        dtEmisao = (f"['{dtEmisao}']").replace("-", "','").replace(":", "','").replace("T", "','")
        sData = eval(dtEmisao)
        dtEmisao = (f"{sData[2]}/{sData[1]}/{sData[0][2:4]}")  

        sChaveNF = xml["nfeProc"]["protNFe"]["infProt"]["chNFe"]
        sVprod = xml["nfeProc"]["NFe"]["infNFe"]["total"]["ICMSTot"]["vProd"]  
        sVtProd = xml["nfeProc"]["NFe"]["infNFe"]["total"]["ICMSTot"]["vNF"]
        sVolume = xml["nfeProc"]["NFe"]["infNFe"]["transp"]["vol"]["qVol"]
        try:
            sPesoB = xml["nfeProc"]["NFe"]["infNFe"]["transp"]["vol"]["pesoB"]
        except:
            sPesoB = "0,01"
        #--Encapsular dados---------------------------------------------------       
        xmlValue = (f"""[
            '{sChaveNF}','{dtEmisao}','{sVolume}','{sPesoB}','{sVprod}','{sVtProd}',
            '{emitEnd}','{emitCid}','{emitNome}','{emitCNPJ}','{emitIE}',
            '{destNome}','{destCNPJ}','{destEnd}','{destCid}','{destIE}','{numerNF}',' ']""")
        
        capsulaXML = eval(xmlValue)
        #--Salve em planilha----------------------------------------------------------     
        # salve_dados(capsulaXML)
    #--Salvar em DataFrame depois exportar excel------------------------------------
        sTabela.loc[len(sTabela.index)] = capsulaXML
    sTabela.to_excel('C:\\Python_Bases\\BaseXML.xlsx', index=False)
    #--End Salve excel---------------------------------------------------------------    
    
#--Salvar Dados na Planilha-----------------------------------------------------------
def salve_dados(capsulaXML):
    workbook = openpyxl.load_workbook(xCaminhPlan) #planilha   
    planilha = workbook.worksheets[0]
    planilha = workbook.active 
    max = planilha.max_row #urtima linha vasia
    if max <= 1:
        planilha.append(titulo)
    planilha.append(capsulaXML)   
    workbook.save(xCaminhPlan)
    
#--Ler nomes dos arquivos na Pasta xCaminhoXML-------------------------------------------
def arquivoXML():
    try:  
        chdir(xCaminhoXML)
        for nomeArquivo in listdir():
            if isfile(nomeArquivo):
                extrair_XML(nomeArquivo)
    except:
        pass
            

arquivoXML()
# salve_DataFrame()


In [ ]:
# display(pd.read_excel('sTabela.xlsx'))
display(sTabela)
